# GigaChat functions agent

В данном ноутбуке рассмотрен пример использования трех функций для созданиня ассистента на базе GigaChat.

### Инициализируем модель GigaChat.

In [40]:
from langchain.chat_models.gigachat import GigaChat
giga = GigaChat(credentials="...", model="GigaChat-Pro", timeout=30)
# giga.verbose = True

### Описываем тул для отправки СМС-сообщений

In [41]:
from langchain.tools import tool
from langchain_core.pydantic_v1 import Field, BaseModel


class SendSmsResult(BaseModel):
    status: str = Field(
        description="Статус отправки сообщения"
    )
    message: str = Field(
        description="Сообщение о результате отправки SMS"
    )

few_shot_examples = [
      {
        "request": "Можешь ли ты отправить SMS-сообщение на номер 123456789 с содержимым 'Привет, как дела?'",
        "params": {
          "recipient": "123456789",
          "message": "Привет, как дела?"
        }
      }
    ]

@tool(few_shot_examples=few_shot_examples)
def send_sms(recipient: str = Field(description="Номер телефона получателя"), 
             message: str = Field(description="Содержимое сообщения")) -> SendSmsResult:
    """Отправить SMS-сообщение"""
    print(f"! send_sms to {recipient}, text: {message}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return SendSmsResult(status="OK", message="Сообщение отправлено!")


### Создаем агента

In [42]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_gigachat_functions_agent

tools = [send_sms]
agent = create_gigachat_functions_agent(giga, tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
)

Теперь с агентом можно пообщаться и давать ему команды

In [44]:
import time

def chat(agent_executor):
    chat_history = [
        SystemMessage(
            content="Ты полезный ассистент. Выполняй команды пользователя, уточняй недостающие данные."
        )
    ]

    while True:
        user_input = input("Клиент: ")
        if user_input == "":
            break
        print(f"User: {user_input}")
        result = agent_executor.invoke(
            {
                "chat_history": chat_history,
                "input": user_input,
            }
        )
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=result["output"]))
        print("\033[93m" + f"Bot: {result['output']}" + "\033[0m")
        time.sleep(0.2)

chat(agent_executor)

User: Отправь поздравление с днем рождения на номер +79992223344
! send_sms to +79992223344, text: С днем рождения!
Bot: Поздравление успешно отправлено на указанный номер.


### Теперь создадим ещё несколько функций для агента
#### Поиск фильмов

In [48]:
from typing import Optional, List


class SearchMoviesResult(BaseModel):
    movies: List[str] = Field(
        description="Список названий фильмов, соответствующих заданным критериям поиска"
    )

few_shot_examples = [
    {"request": "Найди все фильмы жанра комедия", "params": {"genre": "комедия"}}
]

@tool(few_shot_examples=few_shot_examples)
def search_movies(
    genre: Optional[str] = Field(None, description="Жанр фильма"),
    year: Optional[int] = Field(None, description="Год выпуска фильма"),
    actor: Optional[str] = Field(None, description="Имя актера, снимавшегося в фильме"),
) -> SearchMoviesResult:
    """Поиск фильмов на основе заданных критериев"""
    print(f"! search_movies genre {genre}, year: {year}, actor: {actor}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return SearchMoviesResult(movies=["Опенгеймер"])

### Калькулятор расстояний

In [49]:
class TripDistanceResult(BaseModel):
    distance: int = Field(
        description="Расстояние между начальным и конечным местоположением в километрах"
    )

few_shot_examples = [
    {
        "request": "Насколько далеко от Москвы до Санкт-Петербурга?",
        "params": {
          "start_location": "Москва",
          "end_location": "Санкт-Петербург"
        }
      }
]

@tool(few_shot_examples=few_shot_examples)
def calculate_trip_distance(
    start_location: str = Field(description="Начальное местоположение"),
    end_location: str = Field(description="Конечное местоположение"),
) -> TripDistanceResult:
    """Рассчитать расстояние между двумя местоположениями."""
    print(f"! calculate_trip_distance start_location {start_location}, end_location: {end_location}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return TripDistanceResult(distance=650)

In [50]:
tools = [send_sms, search_movies, calculate_trip_distance]
agent = create_gigachat_functions_agent(giga, tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
)

chat(agent_executor)

User: Найди фильмы Нолана за 2023
! search_movies genre description='Жанр фильма' extra={}, year: 2023, actor: Нолан
Bot: Фильм Нолана за 2023 год - "Опенгеймер".
User: Сколько ехать из Москвы в Питер?
! calculate_trip_distance start_location Москва, end_location: Санкт-Петербург
Bot: Из Москвы в Санкт-Петербург примерно 650 километров пути.
